<a href="https://colab.research.google.com/github/EvenSol/NeqSim-Colab/blob/master/notebooks/power/Gas_fired_power_plants.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install neqsim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.3/28.3 MB 51.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 35.4 MB/s eta 0:00:00


# Calculation of ideal air fuel ratio
....

In [14]:
from neqsim import jNeqSim
from neqsim.thermo import TPflash, printFrame

fluid = jNeqSim.thermo.system.SystemSrkEos((273.15 + 25.0), 10.00)
fluid.addComponent("nitrogen", 1.0)
fluid.addComponent("CO2", 1.0)
fluid.addComponent("methane", 92.0)
fluid.addComponent("ethane", 5.0)
fluid.addComponent("propane", 1.0)
fluid.addComponent("i-butane", 0.5)
fluid.addComponent("n-butane", 0.5)
fluid.addComponent("i-pentane", 0.1)
fluid.addComponent("n-pentane", 0.1)
fluid.addComponent("n-hexane", 0.01)
fluid.setMixingRule(2)

TPflash(fluid)

printFrame(fluid)

| 0                  | 1           | 2           | 3   | 4   | 5   | 6               |
|:-------------------|:------------|:------------|:----|:----|:----|:----------------|
|                    | total       | GAS         |     |     |     |                 |
| nitrogen           | 9.88045E-3  | 9.88045E-3  |     |     |     | [mole fraction] |
| CO2                | 9.88045E-3  | 9.88045E-3  |     |     |     | [mole fraction] |
| methane            | 9.09001E-1  | 9.09001E-1  |     |     |     | [mole fraction] |
| ethane             | 4.94022E-2  | 4.94022E-2  |     |     |     | [mole fraction] |
| propane            | 9.88045E-3  | 9.88045E-3  |     |     |     | [mole fraction] |
| i-butane           | 4.94022E-3  | 4.94022E-3  |     |     |     | [mole fraction] |
| n-butane           | 4.94022E-3  | 4.94022E-3  |     |     |     | [mole fraction] |
| i-pentane          | 9.88045E-4  | 9.88045E-4  |     |     |     | [mole fraction] |
| n-pentane          | 9.88045E-4  | 9.8804

#Combustion properties

In [15]:
from neqsim.standards import ISO6976

referenceTemperatureVolume = "15.0" #@param ['0.0', '15.0', '20.0']
referenceTemperatureCombustion = 15.0 #@param ['0.0', '15.0', '20.0']
numberUnit = 'mass' #@param ['volume', 'mass', 'molar']

iso6976 = ISO6976(fluid)
iso6976.setReferenceType(numberUnit)
iso6976.setVolRefT(float(referenceTemperatureVolume))
iso6976.setEnergyRefT(float(referenceTemperatureCombustion))
iso6976.calculate()

GCV = round(iso6976.getValue("SuperiorCalorificValue")/1.0e3,3)
WI = round(iso6976.getValue("SuperiorWobbeIndex")/1.0e3,3)
WIkwh = round(iso6976.getValue("SuperiorWobbeIndex")/1.0e3/3.6,3)
relDens = round(iso6976.getValue("RelativeDensity"),3)

print('GCV '+ str(GCV) + ' MJ/' +numberUnit)
print('WI '+ str(WI), ' MJ/' +numberUnit)
print('WI '+ str(WIkwh), ' kWh/' +numberUnit)
print('Relative density '+ str(relDens), ' [-]')

GCV 52.692 MJ/mass
WI 66.882  MJ/mass
WI 18.578  kWh/mass
Relative density 0.621  [-]


In [16]:
elements_h = 0.0
elements_c = 0.0
sum_hc = 0.0
molmass_hc = 0.0
wtfrac_hc = 0.0

for i in range(fluid.getNumberOfComponents()):
    if fluid.getComponent(i).isHydrocarbon():
        sum_hc = sum_hc + fluid.getComponent(i).getz()
        molmass_hc = molmass_hc + fluid.getComponent(i).getz() * fluid.getComponent(i).getMolarMass()
        elements_c = elements_c + fluid.getComponent(i).getz() * fluid.getComponent(i).getElements().getNumberOfElements("C")
        elements_h = elements_h + fluid.getComponent(i).getz() * fluid.getComponent(i).getElements().getNumberOfElements("H")

if sum_hc == 0:
    print('no hydrocarbons in fluid ')
else:
    wtfrac_hc = molmass_hc / fluid.getMolarMass()
    molmass_hc /= sum_hc
    elements_c /= sum_hc
    elements_h /= sum_hc
    aconst = elements_c + elements_h / 4
    afr = aconst * (32.0 + 3.76 * 28.0) / 1000.0 / molmass_hc * wtfrac_hc
    print('ideal air fluid ratio ', afr)

ideal air fluid ratio  16.231224867401274
